# Entrée des données dans une base de données relationnelle

# table 'population' :
Une table appelée population, contenant la population de chaque pays pour chaque année comprise entre 2012 et la dernière année disponible à la date où vous aurez consulté le site de la FAO. 

Elle devra contenir 4 colonnes : pays, code_pays, annee, population.

In [2]:
# importation de la table population_FAO
import pandas as pd
population = pd.read_csv("population_FAO.csv")

In [19]:
# formatage de la table population
population = population.pivot_table(index=["Zone", "Code zone", "Année"], values=["Valeur"], 
                                    aggfunc=sum, columns = ["Élément"])
population.columns = ['Population totale']
population = population.reset_index()

In [20]:
# renommage des noms de colonne
population.columns = ['pays', 'code_pays', 'annee', 'population']

In [21]:
# 'population' mise à la bonne unité
population['population'] = population['population']*1000

# suppression de l'individu Chine en doublon
population = population.loc[population.pays != 'Chine', :]
population.head()

,pays,code_pays,annee,population
0,Afghanistan,2,2012,30696958.0
1,Afghanistan,2,2013,31731688.0
2,Afghanistan,2,2014,32758020.0
3,Afghanistan,2,2015,33736494.0
4,Afghanistan,2,2016,34656032.0


In [24]:
# exportation de la table sous format .csv
population.to_csv("population.csv", index = False)

# Question 15 : 
Proposez une clé primaire pertinente pour cette table.

population.cle_primaire = je proposerai une clé artificielle

# table 'dispo_alim' :
Une table appelée dispo_alim, contenant pour chaque pays, pour chaque produit, et pour chaque année comprise entre 2012 et la dernière année disponible, les informations suivantes:
la nature du produit (deux valeurs possibles : “animal” ou “végétal”)
disponibilité alimentaire en tonnes
disponibilité alimentaire en Kcal/personne/jour
disponibilité alimentaire de protéines en g/personne/jour
disponibilité alimentaire de matières grasses en g/personne/jour

Elle devra contenir ces colonnes : pays, code_pays, année, produit, code_produit, dispo_alim_tonnes, dispo_alim_kcal_p_j, dispo_prot, dispo_mat_gr .

In [3]:
# importation de la table bilan_2013
bilan_2013 = pd.read_csv("bilan_2013.csv")

In [4]:
# formatage du dataframe principal
bilan_2013 = bilan_2013.pivot_table(values = ["Valeur"], index = ["Code Pays", "Pays", "Code Produit", "Produit", 
                                                                  "Année"],columns = ["Élément"], aggfunc = sum)
bilan_2013.columns = ['Aliments pour animaux', 'Autres Utilisations', 
                      'Disponibilité alimentaire (Kcal/personne/jour)', 
                      'Disponibilité alimentaire en quantité (kg/personne/an)', 
                      'Disponibilité de matière grasse en quantité (g/personne/jour)', 
                      'Disponibilité de protéines en quantité (g/personne/jour)', 'Disponibilité intérieure', 
                      'Exportations - Quantité', 'Importations - Quantité', 'Nourriture', 'Pertes', 
                      'Population totale','Production', 'Semences', 'Traitement', 'Variation de stock']
bilan_2013 = bilan_2013.reset_index()

In [5]:
# mise à l'unité utile des variables suivantes :
vecteur = ['Aliments pour animaux', 'Autres Utilisations', 'Disponibilité intérieure', 
           'Exportations - Quantité', 'Importations - Quantité', 'Nourriture', 'Pertes', 'Production', 
           'Semences', 'Traitement', 'Variation de stock']

# on boucle dans la liste vecteur avec chaque élément de la liste 
for elt in vecteur:
    bilan_2013[elt] *= 1000000
    
# mise à l'unité utile de la variable population :
bilan_2013['Population totale'] *= 1000

In [6]:
# pour remplacer les valeurs 'NaN' par une valeur numérique
bilan_2013 = bilan_2013.fillna(0)

# pour obtenir une valeur sur chacune des lignes de la variable 'Population totale'
for elt in pd.unique(bilan_2013['Pays']):
    bilan_2013.loc[bilan_2013['Pays'] == elt,
                   'Population totale'] = bilan_2013.loc[bilan_2013['Pays'] == elt,'Population totale'].max()
    
# suppression de l'individu Chine en doublon
bilan_2013 = bilan_2013.loc[bilan_2013.Pays != 'Chine', :]

# pour supprimer la valeur 'Population' de l'ensemble de la variable 'Produit'
bilan_2013 = bilan_2013.loc[bilan_2013.Produit != 'Population', :]

In [7]:
# création d'une nouvelle colonne résultante de la mutliplication de la colonne 
#'Disponibilité alimentaire en quantité (kg/personne/an)' avec 'Population totale', le tout divisé par 1 000 
# ( pour être en tonne), le tout multiplé par le nombre de jours, que compte une année
bilan_2013.loc[:, 'Disponibilité alimentaire (tonne/pays/an)'] = bilan_2013['Disponibilité alimentaire en quantité (kg/personne/an)'] * bilan_2013['Population totale']  / 1000

In [8]:
# sélection des colonnes
dispo_alim = bilan_2013[['Pays', 'Code Pays', 'Année', 'Produit', 'Code Produit', 
                         'Disponibilité alimentaire (tonne/pays/an)', 
                         'Disponibilité alimentaire (Kcal/personne/jour)', 
                         'Disponibilité de protéines en quantité (g/personne/jour)',
                         'Disponibilité de matière grasse en quantité (g/personne/jour)']]

In [9]:
# renommage des noms de colonne
dispo_alim.columns = ['pays', 'code_pays', 'annee', 'produit', 'code_produit', 'dispo_alim_tonnes', 
                      'dispo_alim_kcal_p_j', 'dispo_prot', 'dispo_mat_gr']
dispo_alim.head()

,pays,code_pays,annee,produit,code_produit,dispo_alim_tonnes,dispo_alim_kcal_p_j,dispo_prot,dispo_mat_gr
1,Arménie,1,2013,Blé,2511,388796.20,1024.0,30.52,3.60
2,Arménie,1,2013,Orge,2513,0.00,0.0,0.00,0.00
3,Arménie,1,2013,Maïs,2514,89.31,0.0,0.01,0.00
4,Arménie,1,2013,Seigle,2515,357.24,1.0,0.02,0.00
5,Arménie,1,2013,Avoine,2516,1101.49,2.0,0.09,0.03


In [12]:
# exportation de la table sous format .csv
dispo_alim.to_csv("dispo_alim.csv", index = False, sep=";")

# Question 16 : 
Proposez une clé primaire pertinente pour cette table.

dispo_alim_primaire = je proposerai une clé artificielle 

# table 'equilibre_prod' :
Une table appelée equilibre_prod, contenant pour chaque pays, pour chaque produit, et pour chaque année comprise entre 2012 et la dernière année disponible, les quantités suivantes :
disponibilité intérieure
aliments pour animaux
semences
pertes
transformés
nourriture
autres utilisations

Elle devra contenir ces colonnes : pays, code_pays, année, produit, code_produit, dispo_int, alim_ani, semences, pertes, transfo, nourriture, autres_utilisations.

In [14]:
# sélection des colonnes
equilibre_prod = bilan_2013[['Pays', 'Code Pays', 'Année', 'Produit', 'Code Produit', 'Disponibilité intérieure', 
                             'Aliments pour animaux', 'Semences', 'Pertes', 'Nourriture', 'Autres Utilisations', 
                             'Exportations - Quantité']]

In [15]:
# renommage des noms de colonne
equilibre_prod.columns = ['pays', 'code_pays', 'annee', 'produit', 'code_produit', 'dispo_int', 'alim_ani',
                          'semences', 'pertes', 'nourriture', 'autres_utilisations', 'exportation']
equilibre_prod.head()

,pays,code_pays,annee,produit,code_produit,dispo_int,alim_ani,semences,pertes,nourriture,autres_utilisations,exportation
1,Arménie,1,2013,Blé,2511,554000000.0,93000000.0,30000000.0,32000000.0,389000000.0,0.0,1000000.0
2,Arménie,1,2013,Orge,2513,198000000.0,137000000.0,14000000.0,15000000.0,0.0,26000000.0,0.0
3,Arménie,1,2013,Maïs,2514,102000000.0,96000000.0,0.0,7000000.0,0.0,0.0,0.0
4,Arménie,1,2013,Seigle,2515,1000000.0,1000000.0,0.0,0.0,0.0,0.0,0.0
5,Arménie,1,2013,Avoine,2516,6000000.0,4000000.0,0.0,0.0,1000000.0,0.0,0.0


In [16]:
# exportation de la table sous format .csv
equilibre_prod.to_csv("equilibre_prod.csv", index = False)

# Question 17 : 
Proposez une clé primaire pertinente pour cette table.

equilibre_prod = je proposerai une clé artificielle 

# table : sous_nutrition :
Une table appelée sous_nutrition, contenant le nombre de personnes en sous-alimentation pour chaque pays et pour chaque année comprise entre 2012 et la dernière année disponible. 

Elle devra contenir 4 colonnes : pays, code_pays, année, nb_personnes.

In [18]:
# importation de la table sous_nutrition_FAO
sous_nutrition = pd.read_csv("sous_nutrition_FAO.csv")

In [19]:
# sélection des colonnes
sous_nutrition = sous_nutrition[['Zone', 'Code zone', 'Année', 'Valeur']]

In [20]:
# renommage des noms de colonne
sous_nutrition.columns = ['pays', 'code_pays', 'annee', 'nb_personnes']

# pour remplacer les valeurs 'NaN' par une valeur numérique
sous_nutrition = sous_nutrition.fillna(0)

# suppression de l'individu Chine en doublon
sous_nutrition = sous_nutrition.loc[sous_nutrition.pays != 'Chine', :]

# 'nb_personnes' mise à la bonne unité
sous_nutrition['nb_personnes'] = sous_nutrition['nb_personnes']*1000000

# valeurs de la variable'annee' corectement nommées
sous_nutrition.loc[sous_nutrition.annee=='2011-2013', 'annee']=2012
sous_nutrition.loc[sous_nutrition.annee=='2012-2014', 'annee']=2013
sous_nutrition.loc[sous_nutrition.annee=='2013-2015', 'annee']=2014
sous_nutrition.loc[sous_nutrition.annee=='2014-2016', 'annee']=2015
sous_nutrition.loc[sous_nutrition.annee=='2015-2017', 'annee']=2016

sous_nutrition.head()

,pays,code_pays,annee,nb_personnes
0,Afghanistan,2,2012,7200000.0
1,Afghanistan,2,2013,8100000.0
2,Afghanistan,2,2014,9000000.0
3,Afghanistan,2,2015,9900000.0
4,Afghanistan,2,2016,10500000.0


In [21]:
# exportation de la table sous format .csv
sous_nutrition.to_csv("sous_nutrition.csv", index = False)

# Question 18 : 
Vous vous en doutez... proposez encore une fois une clé primaire pertinente pour cette table !

sous_nutrition = je proposerai une clé artificielle 

In [22]:
# importation du module sqlite3
import sqlite3

In [24]:
# création de la base de données 'projet_3'
conn = sqlite3.connect('projet_3.db')